In [1]:
import pandas as pd

# Target population

The goal is predict teacher allocation in this population target:

- Educational stage
    - Kindergarten 
    - Primary school

- Teachaers:
    - Teachers
    - Education Assistant
